![ether](https://media2.giphy.com/media/JTV3ciE3YTDycJXhmq/giphy.gif?cid=ecf05e47e01en4yfdxy8feugp8cfur80fovlpp01u38dssnk&rid=giphy.gif&ct=g)

## NFTs data Analysis

In [1]:
# Import dependancies

import os
import requests
import pandas as pd
import json
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# create variable for api key 
covalent_API_KEY = os.getenv("Covalent-API-KEY")

## Get NFT Token IDS

In [4]:
# Append url for our api
url = "https://api.covalenthq.com/v1"
chain_id = "/1"

# Add search queries to api url
contract_address = "/0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB"
api_key = "/?key=" + covalent_API_KEY

url_tokens = url + chain_id + "/tokens" + contract_address + "/nft_token_ids" + api_key #+ "&format=csv"

# Create variale for our requests data
nft_tokens = requests.get(url_tokens).json()

# Display dataframe
token_IDS = pd.DataFrame(nft_tokens['data']['items'])
token_IDS.head()

,contract_decimals,contract_name,contract_ticker_symbol,contract_address,supports_erc,logo_url,token_id
0,0,CRYPTOPUNKS,Ͼ,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,None,https://logos.covalenthq.com/tokens/0xb47e3cd8...,0
1,0,CRYPTOPUNKS,Ͼ,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,None,https://logos.covalenthq.com/tokens/0xb47e3cd8...,1
2,0,CRYPTOPUNKS,Ͼ,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,None,https://logos.covalenthq.com/tokens/0xb47e3cd8...,2
3,0,CRYPTOPUNKS,Ͼ,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,None,https://logos.covalenthq.com/tokens/0xb47e3cd8...,3
4,0,CRYPTOPUNKS,Ͼ,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,None,https://logos.covalenthq.com/tokens/0xb47e3cd8...,4


## Get NFT Contract Transactions

In [5]:
# Create contract address varibles
contract_address = "/0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB"
token_ID = "/200"

# Create search queries for api
url_transactins = url + chain_id + "/tokens" + contract_address + "/nft_transactions" + token_ID + api_key
nft_transactions = requests.get(url_transactins).json()

# Display data
transactions_df = pd.DataFrame(nft_transactions['data']['items'][0]['nft_transactions'][0])
transactions_df.head()

,block_signed_at,block_height,tx_hash,tx_offset,successful,from_address,from_address_label,to_address,to_address_label,value,value_quote,gas_offered,gas_spent,gas_price,fees_paid,gas_quote,gas_quote_rate,log_events
0,2022-03-17T03:51:37Z,14401717,0x5b0ee618e7eef1baefc510019edb2c9b5962ee1730e4...,6,True,0xc352b534e8b987e036a93539fd6897f53488e56a,None,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CRYPTOPUNKS (Ͼ),0,0.0,400000,48279,60000000000,2896740000000000,8.196203,2829.45752,"{'block_signed_at': '2022-03-17T03:51:37Z', 'b..."
1,2022-03-17T03:51:37Z,14401717,0x5b0ee618e7eef1baefc510019edb2c9b5962ee1730e4...,6,True,0xc352b534e8b987e036a93539fd6897f53488e56a,None,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CRYPTOPUNKS (Ͼ),0,0.0,400000,48279,60000000000,2896740000000000,8.196203,2829.45752,"{'block_signed_at': '2022-03-17T03:51:37Z', 'b..."


## GET Address Balances

In [6]:
# Create account addresses
address = "/msaki.eth"

# Search api to return data
url_bal = url + chain_id + "/address" + address + "/balances_v2" + api_key
balances = requests.get(url_bal).json()

# Display data
balance_df = pd.DataFrame(balances['data']['items'])
balance_df.head()

,contract_decimals,contract_name,contract_ticker_symbol,contract_address,supports_erc,logo_url,last_transferred_at,type,balance,balance_24h,quote_rate,quote_rate_24h,quote,quote_24h,nft_data
0,18,Ether,ETH,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,None,https://www.covalenthq.com/static/images/icons...,None,cryptocurrency,11923112004967972325026,11627900631422460178004,2885.521000,2863.487300,3.440439e+07,3.329635e+07,None
1,18,ApeCoin,APE,0x4d224452801aced8b2f0aebe155379bb5d594381,[erc20],https://logos.covalenthq.com/tokens/1/0x4d2244...,2022-04-25T23:55:43Z,cryptocurrency,4387375327825867762092,4387375327825867762092,19.957893,19.362696,8.756277e+04,8.495141e+04,None
2,18,Bemil.io,BEM,0x3102931098e75111a6f57cdf9a3e68dd007e5dd1,[erc20],https://logos.covalenthq.com/tokens/1/0x310293...,2021-11-06T22:49:45Z,cryptocurrency,10238000000000000000000,10238000000000000000000,0.391597,NaN,4.009175e+03,NaN,None
3,6,USD Coin,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,[erc20],https://logos.covalenthq.com/tokens/1/0xa0b869...,2021-11-20T09:12:25Z,stablecoin,2545889125,2545889125,0.999352,0.993984,2.544238e+03,2.530572e+03,None
4,6,Tether USD,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,[erc20],https://logos.covalenthq.com/tokens/1/0xdac17f...,2022-04-23T15:21:23Z,stablecoin,1142525387,1142525387,1.000139,0.996329,1.142684e+03,1.138331e+03,None


## Get historical portfolio value over time

In [7]:
# Create account addresses 
address = "/msaki.eth"

# Search api for address and portfolio data
url_portfolio = url + chain_id + "/address" + address + "/portfolio_v2" + api_key
portfolio = requests.get(url_portfolio).json()

# Display data
portfolio_df = pd.DataFrame(portfolio['data']['items'][0]['holdings'])

In [8]:
portfolio_df.head()

,timestamp,quote_rate,open,high,low,close
0,2022-04-28T00:00:00Z,2885.5210,"{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'..."
1,2022-04-27T00:00:00Z,2863.4873,"{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'..."
2,2022-04-26T00:00:00Z,2979.3630,"{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'..."
3,2022-04-25T00:00:00Z,2933.9536,"{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'..."
4,2022-04-24T00:00:00Z,2928.9316,"{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'...","{'balance': '11923112004967972325026', 'quote'..."


## Get Market Place data

In [9]:
# Create global varial for marketplace
url_global_view = url + chain_id + "/nft_market" + api_key
global_view = requests.get(url_global_view).json()

# Review data as dataframe
global_view_df = pd.DataFrame(global_view['data']['items'])
global_view_df.head(10)

,chain_id,collection_name,collection_address,volume_wei_24h,volume_quote_24h,avg_volume_wei_24h,avg_volume_quote_24h,contract_deployment_at,market_cap_wei,market_cap_quote,...,floor_price_wei_7d,floor_price_quote_7d,gas_quote_rate,quote_currency,opening_date,first_nft_image_token_id,first_nft_image,first_nft_image_256,first_nft_image_512,first_nft_image_1024
0,1,Rarible,0xd07dc4262bcdbf85190c01c996b4c06a461d2430,1000000000000000000,3.477595e+03,1000000000000000000,3477.5955,2020-05-27T11:49:56Z,6269271254011360000000000000000,2.180199e+16,...,137816704081529000,4.792708e+02,3477.5955,USD,2022-04-03,1,https://cloudflare-ipfs.com/ipfs/QmdU9h6dECWKR...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
1,1,Town Star,0xc36cf0cfcb5d905b8b513860db0cfe63f6cf9f5c,0,0.000000e+00,0,0.0000,2020-06-04T10:10:10Z,35996080896093600000000000,1.251798e+11,...,865000000000000000,3.008120e+03,3477.5955,USD,2022-04-03,None,None,None,None,None
2,1,Fantasy Islands - Sandbox,0x68aea268f92b715e073d2f5170039faa4b94a47b,0,0.000000e+00,0,0.0000,2021-08-20T00:00:55Z,1153800000000000000000000,4.012450e+09,...,7770666666666670000000,2.702324e+07,3477.5955,USD,2022-04-03,1,https://ipfs.io/ipfs/QmPqJyCtpQux4CZ4uVpJ9PdFv...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
3,1,BoredApeYachtClub,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,743000000000000000000,2.583854e+06,185750000000000000000,645963.4000,2021-04-22T03:03:16Z,311400386483699000000000,1.082925e+09,...,118649928843994000000,4.126165e+05,3477.5955,USD,2022-04-03,0,https://ipfs.io/ipfs/QmRRPWG96cmgTn2qSzjwr2qvf...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
4,1,Sandbox's LANDs,0x50f5474724e0ee42d9a4e711ccfb275809fd6d4a,2390000000000000000,8.311453e+03,2390000000000000000,8311.4530,2019-12-04T07:51:55Z,263567420745161000000000,9.165809e+08,...,2330554166666670000,8.104725e+03,3477.5955,USD,2022-04-03,45,https://api.sandbox.game/lands/afc9cb8b-c281-4...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
5,1,Meebits,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,40190000000000000000,1.397646e+05,5023750000000000000,17470.5700,2021-05-03T00:49:32Z,237309568302152000000000,8.252667e+08,...,102961367689247000000,3.580580e+05,3477.5955,USD,2022-04-03,1,http://meebits.larvalabs.com/meebitimages/char...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
6,1,Art Blocks,0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270,161312079990000000000,5.609782e+05,1772660219670330000,6164.5950,2020-12-12T08:59:58Z,213785789663137000000000,7.434605e+08,...,1156779546260830000,4.022811e+03,3477.5955,USD,2022-04-03,3000000,https://api.artblocks.io/image/3000000,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
7,1,MutantApeYachtClub,0x60e4d786628fea6478f785a6d7e704777c86a7c6,711030000000000000000,2.472675e+06,25393928571428600000,88309.8100,2021-08-28T23:59:15Z,180697272256570000000000,6.283920e+08,...,24275317688972800000,8.441973e+04,3477.5955,USD,2022-04-03,0,https://ipfs.io/ipfs/QmURua8bNrAwX76Tp6G9t6Los...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
8,1,Azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,4876178000000000000000,1.695737e+07,35334623188405800000,122879.5200,2022-01-10T05:19:57Z,167638580283958000000000,5.829791e+08,...,41440013247272700000,1.441116e+05,3477.5955,USD,2022-04-03,0,https://azuki-prereveal.s3.us-west-1.amazo

## Get Historical data for NFT collection

In [10]:
collection_address = ""